In [1]:
from google.colab import drive
drive.mount ('/content/drive/')

Mounted at /content/drive/


**pretrained model in arabic**

In [2]:

import pandas as pd


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.9 MB/s eta 0:00:00


In [5]:
from transformers import pipeline
model = pipeline('text-classification', model='Ammar-alhaj-ali/arabic-MARBERT-news-article-classification')

In [7]:
sentences=["""قال إنه بعد صدور قرار وزير المالية بالمرحلة الثانية ( الفرعية الأولى، والفرعية الثانية ) لمنظومة  توحيد أسس ومعايير احتساب ضريبة الأجور والمرتبات ، قامت المصلحة بتنظيم عدة ورش عمل فنية لممولي المرحلة الثانية من تطبيق المنظومة ، تم خلالها شرح المنظومة، والرد على كافة الاستفسارات والتساؤلات الواردة خلال هذه  الورش ، لافتًا إلى أنه تم نشر تسجيل ورشة العمل على قناة اليوتيوب الرسمية لمصلحة الضرائب المصرية ، وكذلك على الموقع الإلكتروني للمصلحة من خلال " دليلك للتعامل مع منظومة توحيد معايير احتساب ضريبة الأجور والمرتبات".

   وأوضح  أن  منظومة توحيد أسس ومعايير احتساب ضريبة الأجور والمرتبات تهدف إلى التخفيف من المخاطر والتكاليف التشغيلية والتراكمات المرتبطة بفحص ضريبة الأجور والمرتبات ، كما تهدف إلى العمل على الحد من وجود فروق الفحص عن طريق زيادة الدقة والالتزام بحساب وسداد الضريبة ، وتخفيف العبء الإداري وتقليل تكلفة المعاملات والحاجة إلى أرشفة المستندات ورقياً.

أشار  إلى أن المنظومة تهدف إلى الحد من فروق الفحص الضريبي من خلال التحقق المسبق للبيانات الواردة من الممول، وكذلك تحقيق العدالة الضريبية بين كافة فئات المجتمع الضريبي وتعزيز الالتزام الضريبي ، قائلًا إن المنظومة تعمل على تمكين جميع موظفي الدولة داخل القطاعات المختلفة من الاطلاع على مفردات مرتباتهم ، ومقدار الضريبة المستحقة بما يُسهم في تحقيق العدالة بين جميع العاملين ، كما أنها تعمل على تحقيق المستهدفات المنشودة بتعزيز حوكمة المنظومة المالية للدولة واستيداء مستحقات الخزانة العامة.


"""]

In [8]:
m=model(sentences)
print(m)

[{'label': 'Finance', 'score': 0.996180534362793}]


In [10]:
m[0]['label']

'Finance'

**trained model**

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
dataset=pd.read_csv("/content/drive/MyDrive/final project_NTI/Data/final data/Combined_files_cleaned_final_v2shuffled.csv")

In [21]:
dataset=dataset[['body', 'category']]
dataset

,body,category
0,Hungarian Prime Minister Viktor Orban revealed...,Arabs and the world
1,The Cairo International Film Festival announce...,Culture and arts
2,"Ahmed Rashid, Governor of Giza, confirmed that...",Provinces
3,"The designated governor of Khartoum, Ahmed Oth...",Arabs and the world
4,The General Administration of Transport and Co...,Accidents
...,...,...
60846,"Today, Friday, March 10, the Drinking Water an...",News
60847,The General Administration of Cairo Traffic as...,Accidents
60848,The Official Gazette published the decisions o...,News
60849,"Alaa Ezz, Secretary General of the Federation ...",Economy


In [22]:
dataset.shape

(60851, 2)

In [23]:
target_category = dataset['category'].unique()
print(target_category)

['Arabs and the world' 'Culture and arts' 'Provinces' 'Accidents'
 'Economy' 'News' 'sports']


In [24]:
dataset['CategoryId'] = dataset['category'].factorize()[0]


In [25]:
dataset

,body,category,CategoryId
0,Hungarian Prime Minister Viktor Orban revealed...,Arabs and the world,0
1,The Cairo International Film Festival announce...,Culture and arts,1
2,"Ahmed Rashid, Governor of Giza, confirmed that...",Provinces,2
3,"The designated governor of Khartoum, Ahmed Oth...",Arabs and the world,0
4,The General Administration of Transport and Co...,Accidents,3
...,...,...,...
60846,"Today, Friday, March 10, the Drinking Water an...",News,5
60847,The General Administration of Cairo Traffic as...,Accidents,3
60848,The Official Gazette published the decisions o...,News,5
60849,"Alaa Ezz, Secretary General of the Federation ...",Economy,4


In [26]:
# Create a new pandas dataframe "category", which only has unique Categories, also sorting this list in order of CategoryId values
category = dataset[['category', 'CategoryId']].drop_duplicates().sort_values('CategoryId')
category

,category,CategoryId
0,Arabs and the world,0
1,Culture and arts,1
2,Provinces,2
4,Accidents,3
5,Economy,4
19,News,5
26,sports,6


In [27]:
dataset['category'].value_counts()

Provinces              9983
Arabs and the world    9872
Accidents              9469
Economy                9198
News                   8905
Culture and arts       8483
sports                 4941
Name: category, dtype: int64

**Preprocessing**

In [28]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import en_core_web_sm
import spacy
import re
import string

In [29]:
!python -m spacy download en_core_web_sm
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm')

2023-05-21 19:15:01.923076: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 44.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
stop_words = set(stopwords.words('english'))

In [31]:
nlp = en_core_web_sm.load()

In [32]:
def remove_punc(news_text):
    result = re.sub(r'[0-9]+', '', news_text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub('', result)
    return text


In [34]:
def remove_stop(the_text):
  # Tokenize the text into words
  words = word_tokenize(the_text)

  # Filter out the stop words from the text
  filtered_words = [word for word in words if not word in stop_words]

  # Join the filtered words into a string
  filtered_text = ' '.join(filtered_words)

  # return the filtered text
  return(filtered_text)


In [35]:
nlp = spacy.load('en_core_web_sm')

nlp = en_core_web_sm.load()


In [36]:
def lemm(text):
    lemme=[]
    for token in nlp(text):
        lemme.append(token.lemma_)

    return " ".join(lemme)

In [37]:
def preprocessing_text(text):
    text = remove_punc(text)
    text = remove_stop(text)
    text = lemm(text)
    return text

In [38]:
dataset['text']=dataset['body'].apply(preprocessing_text)

In [39]:
dataset['text']

0        hungarian Prime Minister Viktor Orban reveal t...
1        the Cairo International Film Festival announce...
2        Ahmed Rashid Governor Giza confirm hospital vi...
3        the designate governor Khartoum Ahmed Othman H...
4        the General Administration Transport Communica...
                               ...                        
60846    today Friday March Drinking Water Wastewater C...
60847    the General Administration Cairo Traffic assig...
60848    the official Gazette publish decision Council ...
60849    Alaa Ezz Secretary General Federation Chambers...
60850    the security service various security director...
Name: text, Length: 60851, dtype: object

In [40]:
data_prprocessed=dataset[['text','category','CategoryId']]
data_prprocessed.to_csv('/content/drive/MyDrive/final project_NTI/Data/final data/preprocessed_final_60.csv')

**Vectorization**

In [41]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize



In [43]:
# Tokenize documents and create TaggedDocuments for Doc2Vec model
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[str(i)]) for i, doc in enumerate(data_prprocessed['text'])]

# Train Doc2Vec model
model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4, epochs=20)

In [44]:
# Obtain vector representations for each document
vectors = [model.docvecs[str(i)] for i in range(len(data_prprocessed))]

<ipython-input-44-c5b1bbefd54e>:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  vectors = [model.docvecs[str(i)] for i in range(len(data_prprocessed))]


In [46]:
# Add vector representations to DataFrame
data_prprocessed['vector'] = vectors
data_prprocessed


,text,category,CategoryId,vector
0,hungarian Prime Minister Viktor Orban reveal t...,Arabs and the world,0,"[0.28605002, 0.4638631, 0.5650606, 0.27922526,..."
1,the Cairo International Film Festival announce...,Culture and arts,1,"[-0.5910453, 0.4221998, 0.5737935, -0.14507538..."
2,Ahmed Rashid Governor Giza confirm hospital vi...,Provinces,2,"[-0.12955907, 0.05349117, 0.19492416, -0.59231..."
3,the designate governor Khartoum Ahmed Othman H...,Arabs and the world,0,"[-0.18898922, 0.39275694, 0.6524691, -0.959934..."
4,the General Administration Transport Communica...,Accidents,3,"[-0.45124438, 0.29736218, 0.21519853, -0.19160..."
...,...,...,...,...
60846,today Friday March Drinking Water Wastewater C...,News,5,"[-0.30507925, -0.69442403, -0.40715426, -0.316..."
60847,the General Administration Cairo Traffic assig...,Accidents,3,"[-0.4835313, 0.47931725, -0.643053, -0.6162939..."
60848,the official Gazette publish decision Council ...,News,5,"[-0.17241241, 0.35425502, 0.33264762, -0.16841..."
60849,Alaa Ezz Secretary General Federation Chambers...,Economy,4,"[-0.20488901, -0.15486458, 0.4753982, -0.89823..."


In [47]:
x=np.array(data_prprocessed['vector'])
y=np.array(data_prprocessed['CategoryId'])

In [48]:
print("X.shape = ",x.shape)
print("y.shape = ",y.shape)

X.shape =  (60851,)
y.shape =  (60851,)


In [49]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

**DecisionTree with doctovec**

In [50]:
# Train decision tree classifier on training set
clf = DecisionTreeClassifier(random_state=42)
clf.fit(list(X_train), y_train)

DecisionTreeClassifier(random_state=42)

In [51]:
# Predict labels for testing set using trained classifier
y_pred = clf.predict(list(X_test))


In [52]:
# Calculate accuracy of predictions
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.4592063100813409


**RandomForest with doctovec**

In [53]:
# Train random forest classifier on training set
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(list(X_train), y_train)


RandomForestClassifier(random_state=42)

In [54]:
# Predict labels for testing set using trained random forest classifier
y_pred_rf = rf.predict(list(X_test))

# Calculate accuracy of predictions for random forest classifier
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy of random forest classifier: {accuracy_rf}")

Accuracy of random forest classifier: 0.7619751869197272


**Logistic Regression with doctovec**

In [55]:
# Train logistic regression classifier on training set
lr = LogisticRegression(random_state=42)
lr.fit(list(X_train), y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [56]:
# Predict labels for testing set using trained logistic regression classifier
y_pred_lr = lr.predict(list(X_test))

# Calculate accuracy of predictions for logistic regression classifier
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Accuracy of logistic regression classifier: {accuracy_lr}")


Accuracy of logistic regression classifier: 0.7415167200723031


#TFI-DF Embedding

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [58]:
preprocess_df=pd.read_csv('/content/drive/MyDrive/final project_NTI/Data/final data/preprocessed_final_60.csv')

In [59]:
preprocess_df.drop('Unnamed: 0', axis=1,inplace=True)

In [60]:
preprocess_df

,text,category,CategoryId
0,hungarian Prime Minister Viktor Orban reveal t...,Arabs and the world,0
1,the Cairo International Film Festival announce...,Culture and arts,1
2,Ahmed Rashid Governor Giza confirm hospital vi...,Provinces,2
3,the designate governor Khartoum Ahmed Othman H...,Arabs and the world,0
4,the General Administration Transport Communica...,Accidents,3
...,...,...,...
60846,today Friday March Drinking Water Wastewater C...,News,5
60847,the General Administration Cairo Traffic assig...,Accidents,3
60848,the official Gazette publish decision Council ...,News,5
60849,Alaa Ezz Secretary General Federation Chambers...,Economy,4


In [61]:
# Preprocess data using TfidfVectorizer
vectorizer = TfidfVectorizer()
x_tf = vectorizer.fit_transform(preprocess_df['text'])

y_tf=preprocess_df['CategoryId']


In [62]:
# Split data into training and testing sets
X_trainn, X_testt, y_trainn, y_testt = train_test_split(x_tf, y_tf, test_size=0.2, random_state=42,shuffle=True, stratify=y_tf)

In [63]:
# Train random forest classifier on training set
rf_tf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_tf.fit(X_trainn, y_trainn)


RandomForestClassifier(random_state=42)

In [64]:
# Predict labels for testing set using trained random forest classifier
y_pred_TF_Random = rf_tf.predict(X_testt)

In [65]:
# Calculate accuracy of predictions
accuracy = accuracy_score(y_testt, y_pred_TF_Random)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9217812833785227


**XGBOOST with TFIDF**

In [66]:
df_preprocessed_edit=pd.read_csv('/content/drive/MyDrive/final project_NTI/Data/final data/preprocessed_final_60.csv')

In [67]:
df_preprocessed_edit

,Unnamed: 0,text,category,CategoryId
0,0,hungarian Prime Minister Viktor Orban reveal t...,Arabs and the world,0
1,1,the Cairo International Film Festival announce...,Culture and arts,1
2,2,Ahmed Rashid Governor Giza confirm hospital vi...,Provinces,2
3,3,the designate governor Khartoum Ahmed Othman H...,Arabs and the world,0
4,4,the General Administration Transport Communica...,Accidents,3
...,...,...,...,...
60846,60846,today Friday March Drinking Water Wastewater C...,News,5
60847,60847,the General Administration Cairo Traffic assig...,Accidents,3
60848,60848,the official Gazette publish decision Council ...,News,5
60849,60849,Alaa Ezz Secretary General Federation Chambers...,Economy,4


In [68]:
x_g=df_preprocessed_edit['text']
y_g=df_preprocessed_edit['CategoryId']

In [69]:
# Split data into training and testing sets
X_trainxg, X_testxg, y_traingxg, y_testgxg = train_test_split(x_g, y_g, test_size=0.05, random_state=42,shuffle=True, stratify=y_g)

In [70]:
# Preprocess data using TfidfVectorizer
vectorizer_tf = TfidfVectorizer(max_features=5000,ngram_range=(1,3))
X_train_gb = vectorizer_tf.fit_transform(X_trainxg)
X_testg_gb = vectorizer_tf.transform(X_testxg )


In [71]:
import xgboost as xgb

In [76]:
xg_M = xgb.XGBClassifier()

In [77]:
xg_M.fit(X_train_gb, y_traingxg)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [78]:
# Make predictions on the testing set
y_pred_xg = xg_M.predict(X_testg_gb)

# Evaluate the accuracy of the model
accuracy_xg = accuracy_score(y_testgxg, y_pred_xg)
print("Accuracy: %.2f%%" % (accuracy_xg * 100.0))

Accuracy: 94.31%


In [79]:
# Get precision, recall, f1 scores

precision, recall, f1score, support = score(y_testgxg, y_pred_xg, average='micro')

print('Test Accuracy Score of Basic xgboost{}'.format(accuracy_xg))

print(f'Precision : {precision}')

print(f'Recall : {recall}')

print(f'F1-score : {f1score}')

Test Accuracy Score of Basic xgboost0.9431482090042721
Precision : 0.9431482090042721
Recall : 0.9431482090042721
F1-score : 0.9431482090042721


In [80]:
from sklearn.metrics import confusion_matrix

In [81]:
# Compute the overall confusion matrix
conf_mat = confusion_matrix(y_testgxg, y_pred_xg )

# Print the overall confusion matrix
print('Overall confusion matrix:')
print(conf_mat)

Overall confusion matrix:
[[469   2   0   0  16   6   1]
 [  1 412   4   0   1   6   0]
 [  1   4 466   5   4  17   2]
 [  1   1   4 464   0   2   2]
 [ 17   0   3   0 429  11   0]
 [ 12   7  21   1  13 389   2]
 [  3   0   1   0   0   2 241]]


In [82]:
# Compute the confusion matrix for each class
for i in range(conf_mat.shape[0]):
    class_conf_mat = confusion_matrix(y_testgxg == i, y_pred_xg == i)
    print(f'Confusion matrix for class {i}:')
    print(class_conf_mat)

Confusion matrix for class 0:
[[2514   35]
 [  25  469]]
Confusion matrix for class 1:
[[2605   14]
 [  12  412]]
Confusion matrix for class 2:
[[2511   33]
 [  33  466]]
Confusion matrix for class 3:
[[2563    6]
 [  10  464]]
Confusion matrix for class 4:
[[2549   34]
 [  31  429]]
Confusion matrix for class 5:
[[2554   44]
 [  56  389]]
Confusion matrix for class 6:
[[2789    7]
 [   6  241]]


#save the model

In [ ]:
import joblib

In [ ]:
joblib.dump(xg, 'trained_model.h5')

['trained_model.h5']

In [ ]:
import pickle

In [ ]:
# Save the trained model as a pickle file
with open('trained_model_xgboost_done.pkl', 'wb') as f:
    pickle.dump(xg, f)

#RandomSearch

In [ ]:
from scipy.stats import randint, uniform

# Define the hyperparameter distributions
param_dist = {
    'max_depth': randint(3, 6),
    'learning_rate': uniform(0.001, 0.1),
    'n_estimators': randint(100, 500)
}

In [ ]:
# Perform random search to find the best hyperparameters
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xg, param_distributions=param_dist, n_iter=10, cv=5)
random_search.fit(X_train_gb, y_traingxg)

# Print the best hyperparameters and the corresponding score
print('Best hyperparameters:', random_search.best_params_)
print('Best score:', random_search.best_score_)